In [1]:
import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal
from torch.distributions import Categorical
import numpy as np
import gym
import os

device = torch.device('cpu')
if(torch.cuda.is_available()):
    device = torch.device('cuda:0')

In [2]:
from torch.utils.tensorboard import SummaryWriter
writer_actor = SummaryWriter('PPO_logs/actor')
writer_critic = SummaryWriter('PPO_logs/critic')
writer_scalars = SummaryWriter('PPO_logs')

/home/zp/anaconda3/envs/DRL/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zp/anaconda3/envs/DRL/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zp/anaconda3/envs/DRL/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/zp/anaconda3/envs/DRL/lib/python3.7/site-packages/tensorflow/

In [3]:
class RolloutBuffer:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []
        
    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]

In [4]:
class Actor(nn.Module):
    '''给出一个分布的均值'''
    def __init__(self, state_dim, action_dim):
        super(Actor, self).__init__()
        self.linear1 = nn.Linear(state_dim, 64)
        self.linear2 = nn.Linear(64, 64)
        self.linear3 = nn.Linear(64, action_dim)
        
    def forward(self, state):
        x = torch.tanh(self.linear1(state))
        x = torch.tanh(self.linear2(x))
        x = torch.tanh(self.linear3(x))
        return x 

In [5]:
class Critic(nn.Module):
    def __init__(self, state_dim):
        super(Critic, self).__init__()
        self.linear1 = nn.Linear(state_dim, 64)
        self.linear2 = nn.Linear(64, 64)
        self.linear3 = nn.Linear(64, 1)
    
    def forward(self, state):
        x = torch.tanh(self.linear1(state))
        x = torch.tanh(self.linear2(x))
        x = self.linear3(x)
        return x

In [6]:
class ActorCritic(nn.Module):
    '''把Actor和Critic封装到一起了，这样可以直接评估，比较简单'''
    def __init__(self, state_dim, action_dim, action_std_init):
        super(ActorCritic, self).__init__()
        
        self.action_dim = action_dim
        self.action_var = torch.full((self.action_dim,), action_std_init**2).to(device) # 把一个标量变成动作维度的向量
        
        self.actor = Actor(state_dim, action_dim)
        self.critic = Critic(state_dim)
        
    def set_action_std(self, new_action_std):
        self.action_var = torch.full((self.action_dim,), new_action_std**2).to(device)
        
    def act(self, state):
        '''输入单个状态，得到动作和log, 是之后要用到选择动作的子函数'''
        action_mean = self.actor(state)           # 一行均值，列数是动作维度，1维向量
        cov_mat = torch.diag(self.action_var)     # 一行方差，可以直接diag
        dist = MultivariateNormal(action_mean, cov_mat)
        action = dist.sample()
        action_logprob = dist.log_prob(action)
        
        return action.detach(), action_logprob.detach()  # [x,x,x], x
    
    def evaluate(self, state, action):
        '''输入一批状态和动作，得到一批评估结果, .log_prob(action)中action不必是这个dist筛出来的'''
        action_mean = self.actor(state)          # 好多行均值，列数是动作维度，2维矩阵
        action_var = self.action_var.expand_as(action_mean)  # 方差也要变成好多行
        cov_mat = torch.diag_embed(action_var)   # 不能直接diag，要用diag_embed
        dist = MultivariateNormal(action_mean, cov_mat)
        
        # 如果动作的dim是1，则需要给他改变一下形状，
        # 因为多维action[batch_size, action_dim]，而单维的action[batch_size]，少了个维度,要变成[bactch_size,1]
        if self.action_dim == 1:
            action = action.reshape(-1, 1)   # [batch_size, 1] 只有第二个维度有，下面的log_prob才好使
        
        action_logprobs = dist.log_prob(action)
        state_values = self.critic(state)
        dist_entropy = dist.entropy()
        
        return action_logprobs, state_values, dist_entropy 

In [7]:
class PPO:
    def __init__(self, env, lr_actor, lr_critic, gamma, K_epochs, eps_clip, actions_std_init=0.6):
        self.env = env
        self.state_dim = env.observation_space.shape[0]
        self.action_dim = env.action_space.shape[0]
        self.max_action = float(env.action_space.high[0])
        
        self.action_std = actions_std_init
        self.gamma = gamma
        self.K_epochs = K_epochs
        self.eps_clip = eps_clip
        self.actions_std_init = actions_std_init
        self.buffer = RolloutBuffer()
        
        # 初始化Actor_Critic 整体网络
        self.policy = ActorCritic(self.state_dim, self.action_dim, self.action_std).to(device)
        self.policy_old = ActorCritic(self.state_dim, self.action_dim, self.action_std).to(device)
        
        # 初始化优化器和损失函数
        # 因为一个policy里面有两个网络，所以优化器的参数和学习率是两个字典组成的列表
        self.optimizer = torch.optim.Adam([
            {'params':self.policy.actor.parameters(), 'lr':lr_actor},
            {'params':self.policy.critic.parameters(), 'lr':lr_critic}
        ])

        self.loss_fn = nn.MSELoss()
        self.summary = {}
    
    def set_action_std(self, new_action_std):
        '''给ActorCritic类设置一下std'''
        self.policy.set_action_std(new_action_std)
        self.policy_old.set_action_std(new_action_std)
 
    def decay_action_std(self, action_std_decay_rate, min_action_std):
        '''随着步数增大， 减小action的方差, 并给两个actor网络设置方差'''
        self.action_std -= action_std_decay_rate
        self.action_std = round(self.action_std, 4)      # 保留四位小数
        if (self.action_std <= min_action_std):
            self.action_std = min_action_std
        self.set_action_std(self.action_std)
    
    def select_action(self, state):
        '''根据状态获取动作，得到的动作是用于与环境交互的， 不用于update'''
        with torch.no_grad():
            state = torch.FloatTensor(state).to(device)
            action, action_log_prob = self.policy_old.act(state)
        
        # 选择动作的同时，把state， action, action_log_prob 加入buffer,都是tensor
        self.buffer.states.append(state)
        self.buffer.actions.append(action)    
        self.buffer.logprobs.append(action_log_prob)
        
        return self.max_action * action.detach().cpu().numpy()
    
    def test_select_action(self, state):
        with torch.no_grad():
            state = torch.FloatTensor(state).to(device)
            action = self.policy_old.actor(state)
            return self.max_action * action.detach().numpy()
         
    def update(self):
        # 先把reward转换为能用的折扣过的rewards
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(self.buffer.rewards), reversed(self.buffer.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)  # 把折扣过的动作重头排列
        # 归一化reward
        rewards = torch.FloatTensor(rewards).to(device)
        rewards = (rewards - rewards.mean())/(rewards.std() + 1e-7)  # [4000]
        
        # buffer里面的这三个东西本来就是tensor，所以纵向连接dim=0也要用tensor的连接方法
        old_states = torch.squeeze(torch.stack(self.buffer.states, dim=0)).detach().to(device)   # [4000, 11]
        old_actions = torch.squeeze(torch.stack(self.buffer.actions, dim=0)).detach().to(device) # [4000, 3]
        old_logprobs = torch.squeeze(torch.stack(self.buffer.logprobs, dim=0)).detach().to(device) # [4000]
        
        # update
        for _ in range(self.K_epochs):
            
            # 用现在的actor和critic，评估旧的states和actions
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)
        #   [4000],   [4000, 1]     [4000]
            
            # 把state_values 变成[batch_size] 和rewards形状一致
            state_values = torch.squeeze(state_values)
            
            # 计算ratio
            ratios = torch.exp(logprobs - old_logprobs.detach())

            # 计算loss
            advantages = rewards - state_values.detach()
            surr1 = ratios *  advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages
            
            # PPO 的最终loss
            loss = -torch.min(surr1, surr2) - 0.01 * dist_entropy + 0.5 * self.loss_fn(state_values, rewards) #[4000]
            loss = loss.mean()
            self.summary['loss'] = loss.item()
            
            # 梯度更新
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        
        # 更新目标网络，用load模型参数的方法来硬更新模型
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        # 清空buffer
        self.buffer.clear()        
        
    
    def save(self, checkpoint_path):
        '''只保存模型参数'''
        torch.save(self.policy_old.state_dict(), checkpoint_path)
        
    def load(self, checkpoint_path):
        self.policy_old.load_state_dict(torch.load(checkpoint_path))
        self.policy.load_state_dict(torch.load(checkpoint_path))

In [8]:
env_name = "Hopper-v2"

max_ep_len = 1000                # 每个回合最大步数
max_train_timesteps = int(3e6)
action_std = 0.6
action_std_decay_rate = 0.05
min_action_std = 0.1
action_std_decay_freq = int(2.5e5)  # 多少步减小一下std
print_freq = max_ep_len * 10        # 多少步打印一下奖励
save_model_freq = int(1e5)          # 多少步保存一下模型

update_timestep = max_ep_len*4  # buffer存4000组数据训练一次
K_epochs = 80                   # update一次更新网络80次

eps_clip = 0.2
gamma = 0.99
lr_actor = 0.0003
lr_critic = 0.001

env = gym.make(env_name)

ppo_agent = PPO(env, lr_actor, lr_critic, gamma, K_epochs, eps_clip, action_std)

directory = 'PPO_weigths' + '/' + env_name + '/'
if not os.path.exists(directory):
      os.makedirs(directory)
checkpoint_path = directory + 'ppo_{}.pth'.format(env_name)  # 模型保存会覆盖这个文件，整套训练下来，就保存一个文件

In [11]:
writer_actor.add_graph(ppo_agent.policy.actor, torch.FloatTensor(env.reset()).unsqueeze(0).cpu())

Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_addmm)
Error occurs, No graph saved


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_addmm)

In [10]:
writer_critic.add_graph(ppo_agent.policy.critic, torch.FloatTensor(env.reset()).unsqueeze(0).cpu())

Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_addmm)
Error occurs, No graph saved


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_addmm)

In [9]:
time_step = 0
i_episod = 0

# printing and logging variables
print_running_reward = 0
print_running_episodes = 0

while time_step <= max_train_timesteps:  # 以最大步数作为终止条件，同之前的第二种训练方法
    state = env.reset()
    current_ep_reward = 0
    ep_step = 0
    for t in range(max_ep_len):
        # env.render()
        action = ppo_agent.select_action(state)  # 已经存入了三个东西到buffer了！
        state, reward, done, _ = env.step(action) # 直接更新下一个状态到当前状态，没有出现next_state
        print(reward)
        
        ppo_agent.buffer.rewards.append(reward)
        ppo_agent.buffer.is_terminals.append(done)
        
        time_step += 1
        current_ep_reward += reward
        ep_step += 1
        
        if time_step % update_timestep == 0:
            ppo_agent.update()
            writer_scalars.add_scalar('loss', ppo_agent.summary['loss'], time_step)
        
        if time_step % action_std_decay_freq == 0:
            ppo_agent.decay_action_std(action_std_decay_rate, min_action_std)
            
        # printing average reward
        if time_step % print_freq == 0:
            # 打印这么些回合的平均回合奖励
            print_avg_reward = print_running_reward / print_running_episodes
            print_avg_reward = round(print_avg_reward, 2)

            print("Episode : {} \t\t Timestep : {} \t\t Average Reward : {}".format(i_episod, time_step, print_avg_reward))

            print_running_reward = 0
            print_running_episodes = 0
        
        if time_step % save_model_freq == 0:
            ppo_agent.save(checkpoint_path)
            
        if done:
            break
    
    writer_scalars.add_scalar('episode_reward', current_ep_reward, i_episod)
    print_running_reward += current_ep_reward
    print_running_episodes += 1
    i_episod += 1

0.9692168405059756
0.9669487207414057
0.9560365384778
0.928918541562083
0.861425704732466
0.8339173599371578
0.8887452619949376
0.8222500927004864
0.7689318846692109
0.8088505831804392
0.7239339140616146
0.513151897334341
0.36242123834211004
0.2510903700941865
0.1988563636095358
0.16133001173218176
0.10835495260077282
0.020746996257344975
-0.08099097351669982
-0.13447217676876308
-0.14427311997112097
-0.2218085859700709
-0.3222943017738342
-0.37131280202786143
-0.3891760664858171
-0.34115730740550954
-0.21812970216395888
-0.17081155891709499
-0.29569667413703854
-0.38787255603480475
-0.42834374026784905
-0.5474023290653689
0.9873064891944064
0.9216688240643816
0.8445520012271817
0.7578689647333008
0.6625725813046951
0.6461090281110403
0.66518405645532
0.671658025732205
0.607996299924844
0.5250046459865143
0.5077374127558331
0.47037155415804655
0.500341349488169
0.6219874177855758
0.6611653901327305
0.704525172664889
0.7063901784472697
0.575294743387078
0.5179902293336688
0.466574312640

# 测试

In [9]:
ppo_agent_new = PPO(env, lr_actor, lr_critic, gamma, K_epochs, eps_clip, action_std)

In [10]:
ppo_agent_new.load(checkpoint_path)

In [11]:

for j in range(100):
    state = env.reset()
    for i in range(100000):
        env.render()
        action = ppo_agent_new.test_select_action(state)
        state, reward, done, _ = env.step(action)
        if done:
            break



Creating window glfw


KeyboardInterrupt: 

# discounted 的rewards求解过程

In [12]:
original_rewards = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
is_terminals =     [False, False, False, True, False, False, True, False, False, False]

In [30]:
rewards = []
discounted_reward = 0
for reward, is_terminal in zip(reversed(original_rewards), reversed(is_terminals)):
    if is_terminal:
        discounted_reward = 0
    discounted_reward = reward + (0.9 * discounted_reward)
    print(discounted_reward)
    rewards.insert(0, discounted_reward)  # 把折扣过的动作倒叙排列

1.0
1.9
2.71
1.0
1.9
2.71
1.0
1.9
2.71
3.439


In [31]:
rewards

[3.439, 2.71, 1.9, 1.0, 2.71, 1.9, 1.0, 2.71, 1.9, 1.0]

# 单维Multivariate分布

In [133]:
action_mean = torch.FloatTensor([0.1, 0.5, 0.6])
action_mean

tensor([0.1000, 0.5000, 0.6000])

In [134]:
var = torch.full((3,), 1.)
var

tensor([1., 1., 1.])

In [135]:
diag = torch.diag(var)
diag

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

In [136]:
dist_single = MultivariateNormal(action_mean, diag)

In [137]:
action = dist_single.sample()
action

tensor([ 0.2345, -1.1521,  0.3124])

In [138]:
dist_single.log_prob(action)

tensor(-4.1720)

# 多维Multivariate分布


## 动作是多维的


In [139]:
actions_mean = [action_mean, action_mean, action_mean, action_mean, action_mean]
actions_mean_tensor = torch.stack(actions_mean, dim=0)  # dim=0, 纵向拼接列表里面的元素
actions_mean_tensor

tensor([[0.1000, 0.5000, 0.6000],
        [0.1000, 0.5000, 0.6000],
        [0.1000, 0.5000, 0.6000],
        [0.1000, 0.5000, 0.6000],
        [0.1000, 0.5000, 0.6000]])

In [144]:
varss = var.expand_as(actions_mean_tensor)
varss

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [145]:
diag_emd = torch.diag_embed(varss)
diag_emd

tensor([[[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]]])

In [146]:
dist_multi = MultivariateNormal(actions_mean_tensor, diag_emd)

In [147]:
dist_multi.sample()

tensor([[ 1.2881,  0.5815, -0.3580],
        [-0.7267,  0.3107,  1.7739],
        [ 0.1329, -0.2787, -1.6822],
        [ 0.9439,  1.5735,  0.6496],
        [ 0.2225,  0.8475,  0.0482]])

In [148]:
dist_multi.log_prob(actions_mean_tensor)

tensor([-2.7568, -2.7568, -2.7568, -2.7568, -2.7568])

In [149]:
dist_multi.entropy()

tensor([4.2568, 4.2568, 4.2568, 4.2568, 4.2568])

# stack函数

In [2]:
env = gym.make('Hopper-v2')
state = torch.FloatTensor(env.reset())
states = [state, state, state, state]

In [3]:
print(state)
print(state.shape)

tensor([ 1.2547e+00,  2.4694e-03, -7.5729e-04,  3.7093e-03, -4.7297e-03,
        -2.5605e-03,  7.4058e-04,  9.3807e-04, -1.6994e-03, -1.6434e-03,
         4.7490e-03])
torch.Size([11])


In [5]:
states

[tensor([ 1.2547e+00,  2.4694e-03, -7.5729e-04,  3.7093e-03, -4.7297e-03,
         -2.5605e-03,  7.4058e-04,  9.3807e-04, -1.6994e-03, -1.6434e-03,
          4.7490e-03]),
 tensor([ 1.2547e+00,  2.4694e-03, -7.5729e-04,  3.7093e-03, -4.7297e-03,
         -2.5605e-03,  7.4058e-04,  9.3807e-04, -1.6994e-03, -1.6434e-03,
          4.7490e-03]),
 tensor([ 1.2547e+00,  2.4694e-03, -7.5729e-04,  3.7093e-03, -4.7297e-03,
         -2.5605e-03,  7.4058e-04,  9.3807e-04, -1.6994e-03, -1.6434e-03,
          4.7490e-03]),
 tensor([ 1.2547e+00,  2.4694e-03, -7.5729e-04,  3.7093e-03, -4.7297e-03,
         -2.5605e-03,  7.4058e-04,  9.3807e-04, -1.6994e-03, -1.6434e-03,
          4.7490e-03])]

In [6]:
old_states = torch.stack(states, dim=0)

In [7]:
old_states.shape

torch.Size([4, 11])

In [25]:
torch.squeeze(old_states).shape

torch.Size([4, 11])

In [29]:
log_prob = torch.tensor(0.2)

In [30]:
log_prob

tensor(0.2000)

In [31]:
log_probs = [log_prob, log_prob, log_prob, log_prob]

In [33]:
torch.stack(log_probs, dim=0).shape

torch.Size([4])